# 크롤링 시작하기

## 함수 시작

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import csv 
import pandas as pd

In [2]:
urldata = pd.read_csv('../data/movie_data/concat.csv', encoding='utf-8')[['영화명','네이버URL']]

In [3]:
urldata.head()

,영화명,네이버URL
0,명량,https://movie.naver.com/movie/bi/mi/basic.nhn?...
1,극한직업,https://movie.naver.com/movie/bi/mi/basic.nhn?...
2,신과함께-죄와 벌,https://movie.naver.com/movie/bi/mi/basic.nhn?...
3,국제시장,https://movie.naver.com/movie/bi/mi/basic.nhn?...
4,어벤져스: 엔드게임,https://movie.naver.com/movie/bi/mi/basic.nhn?...


In [4]:
name = []
code = []

for i in range(0, len(urldata)):
    code.append(urldata['네이버URL'][i].split("=")[-1])
    name.append(urldata['영화명'][i])

In [5]:
print(name, code)

['명량', '극한직업', '신과함께-죄와 벌', '국제시장', '어벤져스: 엔드게임', '겨울왕국 2', '아바타', '베테랑', '괴물', '도둑들', '7번방의 선물', '암살', '범죄도시2', '알라딘', '광해, 왕이 된 남자', '왕의 남자', '신과함께-인과 연', '택시운전사', '태극기 휘날리며', '부산행', '해운대', '변호인', '어벤져스: 인피니티 워', '실미도', '아바타: 물의 길', '어벤져스: 에이지 오브 울트론', '기생충', '겨울왕국', '인터스텔라', '보헤미안 랩소디', '검사외전', '엑시트', '관상', '아이언맨 3', '설국열차', '캡틴 아메리카: 시빌 워', '수상한 그녀', '해적: 바다로 간 산적', '국가대표', '디워', '백두산', '과속스캔들', '탑건: 매버릭', '스파이더맨: 파 프롬 홈', '웰컴 투 동막골', '공조', '트랜스포머 3', '히말라야', '미션임파서블:고스트프로토콜', '스파이더맨: 노 웨이 홈', '트랜스포머: 패자의 역습', '밀정', '최종병기 활', '트랜스포머', '써니', '화려한 휴가', '한산: 용의 출현', '스파이더맨: 홈 커밍', '1987', '베를린', '마스터', '터널', '어벤져스', '내부자들', '인천상륙작전', '공조2: 인터내셔날', '은밀하게 위대하게', '곡성', '범죄도시', '타짜', '좋은 놈, 나쁜 놈, 이상한 놈', '늑대소년', '미녀는 괴로워', '군함도', '미션 임파서블: 폴아웃', '다크 나이트 라이즈', '아저씨', '사도', '전우치', '미션 임파서블: 로그네이션', '킹스맨 : 시크릿 에이전트', '투사부일체', '연평해전', '반지의 제왕 : 왕의 귀환', '인셉션', '레미제라블', '닥터 스트레인지: 대혼돈의 멀티버스', '캡틴 마블', '미션 임파서블 3', '쥬라기 월드: 폴른 킹덤', '청년경찰', '가문의 위기(가문의 영광2)', '숨바꼭질', '덕혜옹주', '더 테러 라이브', '쥬라기 월드',

In [6]:
movie_list = name[:10]
movie_code = code[:10]

# movie_list = ['스즈메의 문단속']
# movie_code = ['215932']

cnt = 10
need_reviews_cnt = cnt
movie_title = []
reviews = []
review_total = []

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for i, code in enumerate(movie_code):
    movie_title.append(movie_list[i])
    url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    score_total = int(soup.find("div",{"class":"score_total"}).find("em").text.replace(",",""))
    max_range = score_total//10 + 2 if score_total%10 != 0 else score_total//10 + 1
    
    review_data=[]
    for page in range(1,max_range):
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        
        #get : request로 url의  html문서의 내용 요청
        html = requests.get(url)
        #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
        soup = BeautifulSoup(html.content,'html.parser')
        #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
        
        if need_reviews_cnt == 0:
            need_reviews_cnt = cnt
            break

        
        reviews = soup.find_all("div",{"class":"score_reple"}) #리스트로 저장해
        for review in reviews:
            sentence = review.find_all("span")
            
            if sentence[0].text != "관람객":
                review_data.append(sentence[0].text.strip())

            elif sentence[1].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
                # review_data.append(sentence[3].text.strip())
                review_data.append(sentence[3].find("a").get('data-src'))

            else:
                review_data.append(sentence[1].text.strip())
                # print(sentence[1].text.strip())

            need_reviews_cnt -= 1       # 나중에 빼기
            

        time.sleep(0.5)

    review_total.append(review_data)
    

review_dict = {
    "영화명" : movie_title,
    "리뷰" : review_total
}

review_df = pd.DataFrame(review_dict)
review_df.to_csv('review_utf8.csv', encoding='utf-8')

# 리뷰 데이터 전처리

## 라이브러리 설치 및 임포트

In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
!pip install kss

In [ ]:
!pip install soynlp

In [7]:
import re
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
from pykospacing import Spacing
from kss import split_sentences
from collections import Counter

## 리뷰 데이터 불러오기

In [43]:
# 리뷰 데이터 전처리
review_data = './review_utf8.csv'
review = pd.read_csv(review_data, encoding='UTF-8')

In [44]:
review.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [45]:
review.head()

,영화명,리뷰
0,명량,['오늘 명량 시사회로 방금 보고 왔음. 단연코 올해의 수작임. 밑에 평론가들 캐릭...
1,극한직업,['일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 부담없이 진짜 코...
2,신과함께-죄와 벌,"['이정재가 홍보 따라다닌 이유를 알겠음 ㅋㅋㅋ 그리고 관심병사 연기 도랐다', '..."
3,국제시장,['영화내내 가족에게 잘해야겠다 생각이 들었다.우리 아버지는 날위해 모든것을바치신사...
4,어벤져스: 엔드게임,"['마블 3000만큼 사랑합니다', 'Avengers Assemble!', ""I'm..."


In [ ]:
# # 문장 토큰화
# # review.iloc[[review['영화명']=='스즈메의 문단속']]
# # review
# # print(' :',kss.split_sentences(text))
# re1 = review[review['영화명']=='스즈메의 문단속']['리뷰']
# print(re1)
# type(re1)
# rev1 = ''.join(s for s in re1)
# print(rev1)
# type(rev1)
# # print(split_sentences(re1))


## 자연어 처리

1. 리뷰에서 영어표현, puncation, 이모티콘은 모두 제거한다. (=한글 표현만 남긴다)

2. 띄어쓰기 변환기를 통해 띄어쓰기가 되어있지 않은 문장들을 제대로 변환한다.

3. 형태소 분석기를 통해 각각의 리뷰를 형태소 단위로 분리한다.

4. 한글자로 되어있는 것 제거

5. 빈도수가 많은 순으로 정렬 후 의미없는 단어(불용어) 제거

6. 최소빈도수 이하의 것들은 제거한다.

7. 잘못 분리 된 의미있는 단어들을 찾아서 사용자 단어 사전에 추가 (ex. 디테일 => 디 + 테일로 분리됨)

### 1) 한글 표현 남기기

In [46]:
# 정규표현식을 통해 한글 단어만 남기고 모두 제거하기
def extract_word(re1):
    hangul = re.compile('[^가-힣]') 
    result = hangul.sub(' ', re1) 
    return result

한글 코드 범위   
ㄱ ~ ㅎ: 0x3131 ~ 0x314e   
ㅏ ~ ㅣ: 0x314f ~ 0x3163   
가 ~ 힣: 0xac00 ~ 0xd7a3

In [47]:
print("Before Extraction : ",review['리뷰'][4])
print("After Extraction : ", extract_word(review['리뷰'][4]))

Before Extraction :  ['마블 3000만큼 사랑합니다', 'Avengers Assemble!', "I'm Ironman.", '아이언맨으로 시작해서 아이언맨으로 끝난다..', '아스가디언즈 오브 갤럭시 벌써부터 기대됨 ㅋㅋㅋㅋ', '히어로물보고 울줄 누가알겠습니까', '인피니티워가 10년을 무너뜨리는 영화라면엔드게임은 10년을 돌아보는 영화입니다', '쥐가다살렸음 ㅋㅋㅋㅋ', '마지막 한타에서 쌌습니다', '10점 안주는 사람들은 마블에 6에서900까지밖에 애정이 없는거 ...']
After Extraction :    마블     만큼 사랑합니다                                          아이언맨으로 시작해서 아이언맨으로 끝난다      아스가디언즈 오브 갤럭시 벌써부터 기대됨         히어로물보고 울줄 누가알겠습니까    인피니티워가   년을 무너뜨리는 영화라면엔드게임은   년을 돌아보는 영화입니다    쥐가다살렸음         마지막 한타에서 쌌습니다      점 안주는 사람들은 마블에  에서   까지밖에 애정이 없는거      


In [48]:
# 전체 데이터에 대해 위에 함수 실행
review['리뷰'] = review['리뷰'].apply(lambda x:extract_word(x))

In [49]:
review

,영화명,리뷰
0,명량,오늘 명량 시사회로 방금 보고 왔음 단연코 올해의 수작임 밑에 평론가들 캐릭...
1,극한직업,일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 부담없이 진짜 코...
2,신과함께-죄와 벌,이정재가 홍보 따라다닌 이유를 알겠음 그리고 관심병사 연기 도랐다 ...
3,국제시장,영화내내 가족에게 잘해야겠다 생각이 들었다 우리 아버지는 날위해 모든것을바치신사...
4,어벤져스: 엔드게임,마블 만큼 사랑합니다 ...
5,겨울왕국 2,올라프의 편요약이 기가맥힙니다 크리스토퍼 뮤비에서 좀 흠칫함 미래가...
6,아바타,영화사인 한 획을 그은 영화 스토리 전개 뻔할지도 몰라요 근데 좋음 스토...
7,베테랑,머야 이거 암살 보다 재미 있네 황정민 유해진 오달수의 연기력도 대단하...
8,괴물,뻔한 스토리로 뻔하지 않은 영화를 만들었다 좀 똑똑한 사람들은 이 영화를 ...
9,도둑들,스토리도 탄탄하고 연기자들 연기도 누구 하나 나쁘지 않았는데 왜 저렇게 별...


### 2) 띄어쓰기 고치기

In [50]:
# 띄어쓰기 고치기
from pykospacing import Spacing
spacing = Spacing()

for i in range(0, len(review)):
    review['리뷰'][i] = spacing(review['리뷰'][i]) 

In [38]:
# # 바뀌었는지 전, 후 확인해보기
# print("Before Fixing : ",review['리뷰'][0])
# print("After Fixing : ", spacing(review['리뷰'][0]))
# print("Before Fixing : ",review['리뷰'][1])
# print("After Fixing : ", spacing(review['리뷰'][1]))
# print("Before Fixing : ",review['리뷰'][2])
# print("After Fixing : ", spacing(review['리뷰'][2]))

Before Fixing :    오늘 명량 시사회로 방금 보고 왔음  단연코 올해의 수작임  밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본장수 미화한다며 입털게 분명  평론가들 하는 짓이라곤 남이 차린 밥상 맛있네 맛없네 까대며       이순신  그는 신이다     군도보다 훨씬 잼있음  극장가서 꼭 보세요      일 시사회로 봤어요냉정하게이건 꼭 봐야해  굿    오늘 아침에 조조로 보고왔습니다   월 일에 해군에 입대하는데  그전에 보기 잘한 영화같습니다  평소에 영화 많이 좋아하고 자주보는 편인데  무지 재미있게 봤습니다  나라 잘지키고 오겠습니다 필승    정말 재미있네요 잘보고 왔습니다 시사회    극찬할 수는 없지만 이 정도면 내용  영상  연출 전부 꽤나 뛰어난 작품이지 않은가  평론가들의 평점  점은 도대체 어떻게 해서 나온 점수인지    블랙펄 같은 배가 나오고 판옥선이 거북선으로 트랜스포머하는 장면이        국가에 버림을 받아도 의리로 지키려는 충신  피를 흘리면서 싸우는 군인들의 투지  두려움을 넘어선 백성들의 의지  소중한 가족들을 잃은 민족의 한     슬픈 명작이었습니다        보는 내내 가슴찡하고 울컥하는 진짜 최고의 영화  한 순간도 놓칠 수 없는 해전씬이 특히 압권      작금의 이시점에 국가가 혼란하고 위태할수록국가를 직간접으로 책임지고 있는 리더에게 무엇이 필요한지 리더의 마음이 한치의 흔들림없이 무엇을 향하고 있어야 하는지를 세치의 혀가 아닌 온몸으로    년전 구국의영웅이        
After Fixing :  오늘 명량 시사회로 방금 보고 왔음 단연코 올해의 수작 임 밑에 평론가들 캐릭터 어쩌고 하는데 류승룡 캐릭터 더 살리면 일본 장수 미화한다며 입털 게 분명 평론가들 하는 짓이라곤 남이 차린 밥상 맛있네 맛없네 까대며 이순신 그는 신이 다 군도 보다 훨씬 잼 있음 극장 가서 꼭 보세요 일 시사회로 봤어요 냉정하게 이건 꼭 봐야 해  굿 오늘 아침에 조조로 보고 왔습니다 월 일에

In [51]:
review.head()

,영화명,리뷰
0,명량,오늘 명량 시사회로 방금 보고 왔음 단연코 올해의 수작 임 밑에 평론가들 캐릭터 어...
1,극한직업,일단 진선규님 진짜 연기를 다양하게 잘하셔서 놀랐어요 오랜만에 부담 없이 진짜 코미...
2,신과함께-죄와 벌,이 정재가 홍보 따라다닌 이유를 알겠음 그리고 관심병사 연기 도랐다 남자 셋이서 안...
3,국제시장,영화 내내 가족에게 잘해야겠다 생각이 들었다 우리 아버지는 날 위해 모든 것을 바치...
4,어벤져스: 엔드게임,마블 만큼 사랑합니다 아이언맨으로 시작해서 아이언맨으로 끝난 다 아스가디언즈 오브 ...


### 3) 형태소 분석

In [52]:
from konlpy.tag import Okt
from konlpy.tag import Kkma

In [53]:
# Okt 모델 사용
okt = Okt()
for i in range(0,len(review)):
    review['리뷰'][i] = okt.morphs(review['리뷰'][i],stem=True)
# " ".join(review['리뷰'].tolist())
# words = okt.morphs(words,stem=True)

In [54]:
review

,영화명,리뷰
0,명량,"[오늘, 명량, 시사회, 로, 방금, 보고, 오다, 단연, 코, 올해, 의, 수작,..."
1,극한직업,"[일단, 진선규, 님, 진짜, 연기, 를, 다양하다, 잘, 하다, 놀라다, 오랜, ..."
2,신과함께-죄와 벌,"[이, 정재, 가, 홍보, 따라다니다, 이유, 를, 알다, 그리고, 관심병사, 연기..."
3,국제시장,"[영화, 내내, 가족, 에게, 잘, 하다, 생각, 이, 들다, 우리, 아버지, 는,..."
4,어벤져스: 엔드게임,"[마블, 만큼, 사랑, 하다, 아이언맨, 으로, 시작, 하다, 아이언맨, 으로, 끝..."
5,겨울왕국 2,"[올라프, 의, 편, 요약, 이, 기, 가, 맥힙니, 다, 크리스토퍼, 뮤비, 에서..."
6,아바타,"[영화사, 인, 하다, 획, 을, 그, 은, 영화, 스토리, 전개, 뻔하다, 모르다..."
7,베테랑,"[머, 야, 이, 거, 암살, 보다, 재미, 있다, 황정민, 유해진, 오달수, 의,..."
8,괴물,"[뻔하다, 스토리, 로, 뻔하다, 않다, 영화, 를, 만들다, 좀, 똑똑하다, 사람..."
9,도둑들,"[스토리, 도, 탄탄하다, 연기자, 들, 연기, 도, 누구, 하나, 나쁘다, 않다,..."


# 크롤링 코드 연습

In [ ]:
movie_list = ['더 퍼스트 슬램덩크', '스즈메의 문단속']
movie_code = ['223800', '215932']

cnt = 10
need_reviews_cnt = cnt
movie_title = []
reviews = []
review_total = []

#page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈 
for i, code in enumerate(movie_code):
    movie_title.append(movie_list[i])
    url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page=1'
    #get : request로 url의  html문서의 내용 요청
    html = requests.get(url)
    #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
    soup = BeautifulSoup(html.content,'html.parser')
    #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
    score_total = int(soup.find("div",{"class":"score_total"}).find("em").text.replace(",",""))
    max_range = score_total//10 + 2 if score_total%10 != 0 else score_total//10 + 1
    
    review_data=[]
    for page in range(1,max_range):
        url = f'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page}'
        #get : request로 url의  html문서의 내용 요청
        html = requests.get(url)
        #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
        soup = BeautifulSoup(html.content,'html.parser')
        #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
        
        if need_reviews_cnt == 0:
            need_reviews_cnt = cnt
            break

        
        reviews = soup.find_all("div",{"class":"score_reple"}) #리스트로 저장해
        for review in reviews:
            sentence = review.find_all("span")
            
            if sentence[0].text != "관람객":
                review_data.append(sentence[0].text.strip())
            elif sentence[1].text == "스포일러가 포함된 감상평입니다. ":
                review_data.append(sentence[2].find('a').get('data-src'))
            else:
                review_data.append(sentence[1].text.strip())
            need_reviews_cnt -= 1       # 나중에 빼기
            print(need_reviews_cnt)
            

        time.sleep(0.5)

    review_total.append(review_data)
    

review_dict = {
    "영화명" : movie_title,
    "리뷰" : review_total
}

review_df = pd.DataFrame(review_dict)
review_df 

#     #한 페이지의 리뷰 리스트의 리뷰를 하나씩 보면서 데이터 추출
#     for review in reviews:
#         sentence = review.find("a",{"class":"report"}).get("onclick").split("', '")[2]
#         #만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
#         if sentence != "":
#             movie = review.find("a",{"class":"movie color_b"}).get_text()
#             score = review.find("em").get_text()
#             review_data.append([movie,sentence,int(score)])
#             need_reviews_cnt-= 1
#     #현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지        
#     if need_reviews_cnt < 0:                                         
#         break
#     #다음 페이지를 조회하기 전 0.5초 시간 차를 두기
#     time.sleep(0.5)
     
# columns_name = ["movie","sentence","score"]
# with open ( "samples.csv", "w", newline ="",encoding = 'utf-8' ) as f:
#     write = csv.writer(f)
#     write.writerow(columns_name)
#     write.writerows(review_data)

In [ ]:
# print(movie_title, len(review_data))
# print(review_data[0])
print((review_total[1][-10:]))
print(len(review_total[0]))


# 

# 맨아래